In [12]:
import pandas as pd
import matplotlib.pyplot as plt

title=pd.read_csv("data/title.akas.tsv",sep = "\t")
name=pd.read_csv("data/name.basics.tsv",sep="\t")
criterion=pd.read_csv("data/criterion.csv",sep=",")



In [15]:
print(title.shape)
print(name.shape)
print(criterion.shape)

(54290886, 8)
(14768112, 6)
(1805, 5)


In [16]:
criterion


,spine,title,director,country,year
0,1.0,Grand Illusion,Jean Renoir,"France,",1937.0
1,2.0,Seven Samurai,Akira Kurosawa,"Japan,",1954.0
2,3.0,The Lady Vanishes,Alfred Hitchcock,"United Kingdom,",1938.0
3,4.0,Amarcord,Federico Fellini,"Italy,",1973.0
4,5.0,The 400 Blows,François Truffaut,"France,",1959.0
...,...,...,...,...,...
1800,NaN,Mangrove,Steve McQueen,"United Kingdom,",2020.0
1801,NaN,The Traveler,Abbas Kiarostami,"Iran,",1974.0
1802,NaN,The Gospel According to Matthew,Pier Paolo Pasolini,"Italy,",1964.0
1803,NaN,Gervaise,René Clément,"France,",1956.0


In [17]:
title

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,\N,\N,original,\N,1
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita,US,\N,imdbDisplay,\N,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
4,tt0000001,5,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
54290881,tt9916852,7,エピソード #3.20,JP,ja,\N,\N,0
54290882,tt9916852,8,Episodio #3.20,ES,es,\N,\N,0
54290883,tt9916856,1,The Wind,\N,\N,original,\N,1
54290884,tt9916856,2,The Wind,DE,\N,imdbDisplay,\N,0


In [13]:
#import pandas as pd

# =========================
# LOAD DATA (IMDb TSV FIXES)
# =========================
import pandas as pd

names_df = pd.read_csv(
    "data/name.basics.tsv",
    sep="\t",
    na_values="\\N",
    engine="python",
    quoting=3,          # csv.QUOTE_NONE
    on_bad_lines="skip" # skip malformed rows
)

akas_df = pd.read_csv(
    "data/title.akas.tsv",
    sep="\t",
    na_values="\\N",
    engine="python",
    quoting=3,
    on_bad_lines="skip"
)

criterion_df = pd.read_csv("data/criterion.csv")



# =========================
# BUILD CRITERION LOOKUP
# =========================
#criterion_ids = set(criterion_df["tconst"].astype(str))
# auto-detect the IMDb ID column
#possible_cols = ["tconst", "titleId", "imdb_id", "imdbID", "const"]
#id_col = next(c for c in possible_cols if c in criterion_df.columns)

#criterion_ids = set(criterion_df[id_col].astype(str))
# =========================
# BUILD CRITERION LOOKUP (SAFE)
# =========================
possible_cols = ["tconst", "titleId", "imdb_id", "imdbID", "const"]

matches = [c for c in possible_cols if c in criterion_df.columns]

if not matches:
    raise ValueError(
        f"No IMDb ID column found in criterion_df. "
        f"Available columns: {criterion_df.columns.tolist()}"
    )

id_col = matches[0]
criterion_ids = set(criterion_df[id_col].astype(str))


# =========================
# FILTER NAME.BASICS
# Keep people with at least one Criterion title
# =========================
def has_criterion_title(known_for):
    if pd.isna(known_for):
        return False
    return any(t in criterion_ids for t in known_for.split(","))

names_filtered = names_df[
    names_df["knownForTitles"].apply(has_criterion_title)
].copy()


# =========================
# FILTER TITLE.AKAS
# Keep only Criterion titles
# =========================
akas_filtered = akas_df[
    akas_df["titleId"].isin(criterion_ids)
].copy()


# =========================
# OPTIONAL: ORIGINAL TITLES ONLY
# =========================
akas_filtered = akas_filtered[
    akas_filtered["isOriginalTitle"] == 1
]


# =========================
# EXPLODE KNOWNFORTITLES
# =========================
names_long = (
    names_filtered
    .assign(tconst=names_filtered["knownForTitles"].str.split(","))
    .explode("tconst")
)

names_long = names_long[
    names_long["tconst"].isin(criterion_ids)
]


# =========================
# MERGE PEOPLE ↔ TITLES
# =========================
final_df = names_long.merge(
    akas_filtered,
    left_on="tconst",
    right_on="titleId",
    how="inner"
)


# =========================
# SELECT CLEAN COLUMNS
# =========================
final_df = final_df[
    [
        "nconst",
        "primaryName",
        "primaryProfession",
        "tconst",
        "title",
        "region",
        "language"
    ]
]

print(final_df.head())



ValueError: No IMDb ID column found in criterion_df. Available columns: ['spine', 'title', 'director', 'country', 'year']

In [ ]:
import pandas as pd

# =========================
# LOAD DATA
# =========================
names_df = pd.read_csv(
    "data/name.basics.tsv",
    sep="\t",
    na_values="\\N",
    engine="python",
    quoting=3,
    on_bad_lines="skip"
)

akas_df = pd.read_csv(
    "data/title.akas.tsv",
    sep="\t",
    na_values="\\N",
    engine="python",
    quoting=3,
    on_bad_lines="skip"
)

criterion_df = pd.read_csv("data/criterion.csv")


# =========================
# NORMALIZE TITLES (CRITERION ↔ AKAS)
# =========================
criterion_titles = (
    criterion_df["title"]
    .astype(str)
    .str.lower()
    .str.strip()
)

akas_df["title_norm"] = (
    akas_df["title"]
    .astype(str)
    .str.lower()
    .str.strip()
)


# =========================
# MATCH CRITERION TITLES TO IMDb (AKAS)
# =========================
akas_filtered = akas_df[
    akas_df["title_norm"].isin(criterion_titles)
].copy()

# keep original titles only
akas_filtered = akas_filtered[
    akas_filtered["isOriginalTitle"] == 1
]

# collect matched IMDb IDs
criterion_tconsts = set(akas_filtered["titleId"].astype(str))


# =========================
# FILTER NAME.BASICS
# =========================
def has_criterion_title(known_for):
    if pd.isna(known_for):
        return False
    return any(t in criterion_tconsts for t in known_for.split(","))

names_filtered = names_df[
    names_df["knownForTitles"].apply(has_criterion_title)
].copy()


# =========================
# EXPLODE KNOWNFORTITLES
# =========================
names_long = (
    names_filtered
    .assign(tconst=names_filtered["knownForTitles"].str.split(","))
    .explode("tconst")
)

names_long = names_long[
    names_long["tconst"].isin(criterion_tconsts)
]


# =========================
# MERGE PEOPLE ↔ TITLES
# =========================
final_df = names_long.merge(
    akas_filtered,
    left_on="tconst",
    right_on="titleId",
    how="inner"
)


# =========================
# SELECT CLEAN COLUMNS
# =========================
final_df = final_df[
    [
        "nconst",
        "primaryName",
        "primaryProfession",
        "tconst",
        "title",
        "region",
        "language"
    ]
]

print(final_df.head())



In [ ]:
final_df.head()